In [34]:
import random
import highcharts
import pandas
import sympy
import numpy
import itertools
import collections
from typing import Dict
from sympy.abc import a,b,c,d,x,y,z
import sympy.parsing.sympy_parser as sympy_parser
from sympy.utilities.lambdify import lambdify
import scipy.optimize
import matplotlib.pyplot as plt
import sklearn.metrics

sympy.init_printing()

def optimize(data: pandas.DataFrame, formula: sympy.Expr, y: sympy.Symbol = sympy.Symbol('y'), w0: Dict[sympy.Symbol, float]={}):
    symbols = set(formula.free_symbols)
    if y in symbols:
        raise Exception(f"Symbol {y} used both as output and as part of the formula")
    w = symbols - set([sympy.Symbol(c) for c in data.columns])
    x = symbols - w - set([y])
    
    w = sorted(w, key=lambda a: a.name)
    x = sorted(x, key=lambda a: a.name)
    
    #print(f'x={x}, y={y}, w={w}, cols={data.columns}')
    
    xdata = data.as_matrix(columns = [x.name for x in x])
    ydata = data.as_matrix(columns = [y.name])
    
    formula_lambda = lambdify(x + w, formula, modules=['numpy'])
    differentials_lambda = lambdify(x + w, sympy.Array([sympy.diff(formula, s) for s in w]), modules=['numpy'])
    
    #jac_func = lambdify(x + w, formula)
    def f(x, *w):
        x = [
            x[:, i]
            for i in range(x.shape[1])
        ]
        ret = formula_lambda(*x, *w)
        ret = list(ret)
        #print(f'eval - X={x}, W={w}, ret={ret}')
        return ret
    
    def jac(x, *w):
        nrow, ncol = x.shape
        x = [
            x[:, i]
            for i in range(ncol)
        ]
        ret = numpy.column_stack([
            d if isinstance(d, numpy.ndarray) else numpy.repeat(d, nrow)
            for d in differentials_lambda(*x, *w)
        ])
        #print(f'jac - X={x}, W={w}, ret={ret}')
        return ret
    
    res_opt, res_cov = scipy.optimize.curve_fit(
        f = f,
        jac = jac,
        xdata = xdata,
        ydata = ydata.ravel(),
        p0 = [w0.get(s, random.uniform(0.1, 0.9)) for s in w],
        maxfev = 5000,
    )
    res_err = numpy.sqrt(numpy.diag(res_cov))
       
    r2 = sklearn.metrics.r2_score(f(xdata, *res_opt), ydata.ravel())
    
    #x_test = numpy.column_stack([numpy.linspace(0, xdata.max())])
    #y_test = f(x_test, *res_opt)
    #plt.plot(xdata.ravel(), ydata.ravel(), 'o', markersize=10)
    #plt.plot(x_test, y_test, '-')#, color=color, label=u"%dºC (fitted model, R²=%.1f%%, params=[%s])" % (temp, R2*100, ', '.join(["%.2f" % (x) for x in res.x])))
    #plt.show()
    
    opt_formula = formula.subs(zip(w, res_opt))
    
    return {
        'formula': opt_formula,
        'func': lambdify(x, opt_formula, modules=['numpy']),
        'params': {
            sym: [opt, err]
            for opt, err, sym in zip(res_opt, res_err, w)
        },
        'R²': r2
    }
    

In [36]:
formulas = {
    'Linear':           sympy_parser.parse_expr('k * C'),
    'Langmuir':         sympy_parser.parse_expr('(q_m * C) / (k_L + C)'),
    'Freundlinch':      sympy_parser.parse_expr('k_F * C ** n_F'),
    'Redlich-Peterson': sympy_parser.parse_expr('(k₀ * C) / (1 + k₁*C**k₂)'),
}

In [185]:
class DataframeConv:
    def __init__(self, nrows=20, temperatures=[60,50,40,30], vars=['C₀', 'C', 'q']):
        self.nrows = nrows
        self.temperatures = temperatures
        self.vars = vars
        
    def empty_df(self):
        cols = [
            f'{temp}-{var}'
            for temp, var in itertools.product(self.temperatures, self.vars)
        ]
        return pandas.DataFrame(columns=cols, data = self.nrows * [len(cols) * [None]])
    
    def split_df(self, df):
        ret = {}
        for temp in self.temperatures:
            old_cols = [
                f'{temp}-{var}'
                for var in self.vars
            ]
            slice = df[old_cols].rename(index=str, columns=dict(zip(old_cols,self.vars)))
            slice = slice.applymap(lambda x: x.replace(',', '.') if isinstance(x, str) else x)
            slice = slice.apply(pandas.to_numeric)
            slice = slice.dropna()
            ret[temp] = slice
        return ret

In [13]:
import ipywidgets as widgets
from jupyter_scisheets_widget import scisheets_widget

class RegressionPanel:
    def __init__(self, name: str, formula: sympy.Expr):
        self.name = name
        self.formula = formula
        self._widget_formula = widgets.Label(f'{name}: ${sympy.latex(self.formula)}$')
        self._widget_coefs = widgets.Output()
        self._widget = widgets.VBox([self._widget_formula, self._widget_coefs])

    def data_updated(self, data):
        regressions = []
        series = []
        colors = iter(['#2f7ed8', '#0d233a', '#8bbc21', '#910000', '#1aadce', '#492970', '#f28f43', '#77a1e5', '#c42525', '#a6c96a'])
        symbols = iter(["circle", "square", "diamond", "triangle", "triangle-down"])
        for temp, df in data.items():
            try:
                optimization = optimize(
                    df,
                    self.formula,
                    sympy.Symbol('q')
                )
                regression = collections.OrderedDict([
                    ('Temperature', temp),
                    ('R²', optimization['R²'])
                ])
                for name, value in optimization['params'].items():
                    regression[f'{name}'] = value[0]
                    regression[f'{name}±'] = value[1]
                regressions.append(regression)
                
                color = next(colors)
                symbol = next(symbols)
                
                linkedTo = f'{temp}ºC'
                series.append({
                    'id': f'{temp}',
                    'name':  f'{temp}ºC', 
                    'type':  'line', 
                    'data':  [],
                    'marker': {
                        'symbol': symbol,
                        'enabled': True
                    },
                    'color': color,
                })
                
                series.append({
                    'name':  f'{temp}ºC - Experiments', 
                    'type':  'scatter', 
                    'data':  df.as_matrix(['C', 'q']).tolist(),
                    'color': color,
                    'marker': {
                        'symbol': symbol,
                        'enabled': True
                    },
                    'linkedTo': f'{temp}',
                })
                
                reg_x = numpy.linspace(0, 1.1 * max(df.C), num=200)
                reg_y = optimization['func'](reg_x)
                series.append({
                    'name':  f'{temp}ºC - {self.name} Regression', 
                    'type':  'line', 
                    'data':  pandas.DataFrame({'x':reg_x, 'y':reg_y}).as_matrix().tolist(),
                    'color': color,
                    'marker': {
                        'symbol': symbol,
                        'enabled': False
                    },
                    'linkedTo': f'{temp}',
                })
            except:
                raise
                pass
            
        self._widget_coefs.clear_output()
        with self._widget_coefs:
            display(pandas.DataFrame(regressions))
       
            chart = highcharts.Highchart()
            chart.set_dict_options({
                'title': {
                    'text': f'{self.name} Regression'
                },
                'xAxis': {
                    'labels': {
                        'format': '{value}',
                    },
                    'title': {
                        'text': 'C* (g/L)',
                    },                   
                    'min': 0,
                    'gridLineWidth': 0,
                    'lineWidth': 1,
                    'tickWidth': 1,
                    'tickLength': 10,
                },
                'yAxis': {
                    'labels': {
                        'format': '{value}',
                    },
                    'title': {
                        'text': 'q* g/g',
                    },
                    'min': 0,
                    'gridLineWidth': 0,
                    'lineWidth': 1,
                    'tickWidth': 1,
                    'tickLength': 10,
                },
            })
            for serie in series:
                chart.add_data_set(**serie)
            display(chart)
  

In [37]:
df_parser = DataframeConv()

tabs = [
    RegressionPanel(name, formula)
    for name, formula in formulas.items()
]

def update_all_tabs(data):
    split_df = df_parser.split_df(data)
    for x in tabs:
        x.data_updated(split_df)
tbl = scisheets_widget.HandsonDataFrame(df_parser.empty_df(), onchange=update_all_tabs)

tab_names = ['Table'] + [tab.name for tab in tabs]
children = [tbl._widget] + [tab._widget for tab in tabs]
tab = widgets.Tab()
tab.children = children
for i in range(len(children)):
    tab.set_title(i, tab_names[i])
display(tab)


/opt/conda/lib/python3.6/site-packages/numpy/__init__.py:1: RuntimeWarning: divide by zero encountered in power
  """
/opt/conda/lib/python3.6/site-packages/numpy/__init__.py:1: RuntimeWarning: divide by zero encountered in power
  """


In [6]:
import ast
df = DataframeConv(nrows=3).empty_df()
model_data = df.to_json(orient='split')
#ast.literal_eval(model_data)
#df
model_data

'{"columns":["60-C\\u2080","60-C","60-q","50-C\\u2080","50-C","50-q","40-C\\u2080","40-C","40-q","30-C\\u2080","30-C","30-q"],"index":[0,1,2],"data":[[null,null,null,null,null,null,null,null,null,null,null,null],[null,null,null,null,null,null,null,null,null,null,null,null],[null,null,null,null,null,null,null,null,null,null,null,null]]}'

In [128]:
pandas.DataFrame({'x':[1,2,3], 'y':[1,2,3]})

,x,y
0,1,1
1,2,2
2,3,3
